In [1]:
import semiolog as slg

In [2]:
semiotic = slg.Cenematic("hf_tokenizers")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using custom data configuration corpus-8e1a03b279e6deba
HF google storage unreachable. Downloading and preparing it from source


100%|██████████| 3/3 [00:00<00:00, 421.95it/s]


Dataset text downloaded and prepared to /Users/Gianni/.cache/huggingface/datasets/text/corpus-8e1a03b279e6deba/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 197.94it/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
sent = semiotic.corpus.train["text"][45]

In [4]:
input = semiotic.syntagmatic.bert_tokenizer(sent)
input

{'input_ids': [295, 329, 10, 1045, 233, 2344, 8, 1, 4961, 6482, 19, 57, 1, 8, 3369, 713, 2477, 2848], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
semiotic.syntagmatic.bert_tokenizer.decode(
    input["input_ids"]
    )

'thereis still the development time task of [UNK] thecorrect root for a [UNK] of approximately 7 0000 words'

In [10]:
input = semiotic.paradigmatic.bert_tokenizer(semiotic.corpus.train["text"][:5], verbose = False)
input

{'input_ids': <tf.Tensor: shape=(5, 25), dtype=int32, numpy=
array([[  22, 3991, 5634,  293, 3325, 9673, 3992,   11,   57, 2818,   43,
        3326,   12, 1557,  307,  425, 2207, 1576,  223,  162, 6404,   62,
           1, 2716, 4806],
       [  57,    1, 1960, 1739,  448, 6407, 2081,  339, 6408, 6409,    1,
        4909,   83,   22,   41,  271,    1,  736,    1,  706,    1,   53,
         468,    0,    0],
       [   1,    1,    7, 1560, 3993,  859,  280,  922, 1212,  112,   52,
        6411,  230, 8069, 1766,    1,  413,    0,    0,    0,    0,    0,
           0,    0,    0],
       [ 518,    6,   61,    1,  359,  299,    9,    1,    1,  995, 2910,
          33,  469,  161,  122, 2210,   24,  252,  472,   31,    1,  995,
        2910,   33,    0],
       [ 425,   42, 1561,  401,  360,    9,    1, 6473,   20,    1,    9,
           1,  737,   10, 2459,  519,    0,    0,    0,    0,    0,    0,
           0,    0,    0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(5, 25), dty

In [3]:
semiotic.paradigmatic.build()

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as the 'labels' key of the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


SLG: Compiling model
SLG: Tokenizing dataset...


100%|██████████| 16/16 [00:00<00:00, 1142.53ba/s]


SLG: Building train set
SLG: Building validation set
SLG: Starting training...

Epoch 1/2
104/272 [==========>...................] - ETA: 4:31 - loss: 8.7613

KeyboardInterrupt: 